In [1]:
from tqdm.notebook import tqdm
from api.config.db_config import get_db_connection
import random
import json
import re

from ollama import Client

client = Client(host="http://127.0.0.1:11434")
with open("sdg_label_prompt.md", "r") as f:
    prompt_template = f.read()
        
conn = get_db_connection()

def get_all_patents_number():
    # Fetch the patent data from the database
    patent = {}
    fetch_patent_query = """
    SELECT patent.number, en_abstract
    FROM patent
    WHERE en_abstract IS NOT null
    """

    cursor = conn.cursor()
    cursor.execute(fetch_patent_query)
    for row  in cursor.fetchall():
        patent[row[0]] = row[1]
    cursor.close()
    return patent


def get_description(patent_number):
    descriptions = []
    fetch_description_query = """
    SELECT description_number, description_text
    FROM patent_description
    WHERE patent_number = %s;
    """
    cursor = conn.cursor()
    cursor.execute(fetch_description_query, (patent_number,))
    fetchall = cursor.fetchall()
    for description in fetchall:
        descriptions.append({
            "description_number": description[0],
            "description_text": description[1]
        })
    cursor.close()
    return descriptions

In [2]:
patent = get_all_patents_number()
random_patent = dict(random.sample(list(patent.items()), 6000))

In [3]:
output_file_path = "../src/ai/testsets/dataset.jsonl"
print(f"Début de la création du dataset : {output_file_path}")

# Ouvrir le fichier en mode écriture ('w')
# Il est bon d'utiliser encoding='utf-8' pour les fichiers JSON.
with open(output_file_path, "w", encoding="utf-8") as f_out:
    for key, abs in tqdm(random_patent.items()):
        descriptions = get_description(key)
        patent_text = f"{abs}\n"
        for description in descriptions:
            patent_text += f"{description["description_number"]}: {description["description_text"]}\n"
        
        words_in_text = patent_text.split()
        words_in_text_cut = words_in_text[:3000]
        patent_text_cut = " ".join(words_in_text_cut)
        f_out.write(json.dumps({key: patent_text_cut}, ensure_ascii=False) + "\n")

Début de la création du dataset : ../src/ai/testsets/dataset.jsonl


  0%|          | 0/6000 [00:00<?, ?it/s]